# Introduction: Programming Lab 2

## Basics of Huggingface

---

### 📚 **과목 정보**
- **과목명**: 산업텍스트 애널리틱스 (Industrial Text Analytics)
- **학과**: 서울대학교 산업공학과
- **교수**: 강필성 교수님
- **담당조교**: 천재원

### 🎯 **실습 개요**
- **실습 제목**: Basics of Huggingface
- **실습 키워드**: 
  - Open-source Large Language Model (LLM)
  - Huggingface Transformers
  - Natural Language Processing (NLP)

### 📋 **실습 목표**
- Huggingface Transformers의 기본 개념과 주요 구성 요소를 이해
- 다양한 NLP 모델(Encoder / Decoder)을 다룰 수 있는 기반 함양
- Huggingface Transformers 라이브러리의 구조와 작동 원리를 익혀, 이후 Fine-tuning이나 커스텀 학습으로 확장(실습 Part. 3, 4) 준비
- Tokenizer, 모델 클래스, Forward 등 파이프라인의 핵심 단계를 직접 실행하며 이해


### 🗓️ **실습 계획**
1. Huggingface 개요
- Huggingface와 Open-source LLM 생태계의 위치 및 역할
- Transformers 라이브러리와 관련 서브 패키지(datasets, evaluate 등) 소개
2. 기본 코드 구조 이해
- 모델 불러오기
- 사전 학습 모델과 체크포인트 개념
3. Tokenizer 사용법
- 토크나이징 과정 및 주요 옵션
- 토크나이저와 모델의 연결 방식
4. 대표적인 모델 클래스 소개
- SequenceClassification, CausalLM 등 태스크별 클래스 구조
- forward 메서드의 주요 파라미터와 반환값 분석
5. 추론 및 결과 확인
- 로짓(logits) 해석 및 후처리(softmax, argmax 등)

### 📖 **참고 자료**
- Huggingface 공식 문서(https://huggingface.co/docs)
- Transformers GitHub Repository(https://github.com/huggingface/transformers)
- Huggingface Course(https://huggingface.co/learn/llm-course/chapter1/1)

# **환경 설정**
실습을 시작하기 전에 필요한 라이브러리를 설치하고 import합니다.

In [ ]:
# pip install needed
# %pip install transformers datasets evaluate accelerate scikit-learn -qU
# %pip install -U "huggingface_hub[cli]"

In [ ]:
# CLI 창에서 반드시 Huggingface 로그인 필요
# hf auth login

In [ ]:
# import
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn.functional as F
import transformers

# 버전 확인
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# **실습 시작**

> **💡 참고사항**: 
> - 각 셀을 순서대로 실행해주세요
> - 에러가 발생할 경우 담당 조교에게 문의하세요
> - 실습 중 궁금한 점이 있으면 언제든 질문해주세요

---
# **1. Huggingface 개요**

> **학습 목표**:
> - Huggingface와 Open-source LLM 생태계의 위치 및 역할 이해
> - Transformers 라이브러리와 관련 서브 패키지 이해

## 1.1 Huggingface와 Open-source LLM 생태계

### **Huggingface의 역할**

**Huggingface**는 현재 Open-source LLM 생태계의 중심에 있습니다:

1. **🤗 Hub**: 모델과 데이터셋의 중앙 저장소
   - 500,000+ 개의 모델
   - 100,000+ 개의 데이터셋
   - Git 기반 버전 관리

2. **🔧 Transformers Library**: 통합 API
   - PyTorch, TensorFlow, JAX 지원
   - 동일한 인터페이스로 다양한 모델 사용

3. **🌍 Community**: 활발한 오픈소스 커뮤니티
   - 연구자, 개발자, 기업의 협업
   - 최신 연구 결과의 빠른 구현

### **LLM 생태계에서의 위치**

```
연구 논문 → Huggingface 구현 → 커뮤니티 사용 → 산업 적용
```

- OpenAI (GPT-oss), Google (BERT, T5), Meta (LLaMA) 등의 모델을 통합 관리
- 표준화된 인터페이스로 모델 간 전환 용이

## 1.2 Transformers 라이브러리와 관련 서브 패키지

Huggingface는 여러 상호 연결된 라이브러리들로 구성되어 있습니다:

In [ ]:
# Huggingface 생태계 주요 라이브러리들
print("🤗 Huggingface Ecosystem Libraries:\n")

libraries = {
    "transformers": "모델과 토크나이저를 위한 메인 라이브러리",
    "datasets": "데이터셋 로딩과 처리",
    "evaluate": "모델 평가 메트릭",
    "accelerate": "분산 학습 및 mixed precision",
    "hub": "모델 허브 인터페이스",
}

for lib, desc in libraries.items():
    print(f"📦 {lib:15} : {desc}")

print("\n" + "="*60)

In [ ]:
# 각 라이브러리 간단 예제
from transformers import pipeline
from datasets import load_dataset
import evaluate

print("1. Transformers - Pipeline 예제:")
classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", device='cpu')
result = classifier("Huggingface makes NLP easy!")
print(f"   모델: {classifier.model.name_or_path}")
print("   입력: Huggingface makes NLP easy!")
print(f"   결과: {result}\n")
print("--------------------------------")

print("2. Datasets - 데이터셋 로딩 예제:")
# 작은 샘플 데이터셋 로드
dataset = load_dataset("glue", "sst2", split="train[:5]")
print(f"   데이터셋 크기: {len(dataset)}")
print(f"   첫 번째 샘플: {dataset[2]}\n")
print("--------------------------------")

print("3. Evaluate - 평가 메트릭 예제:")
accuracy = evaluate.load("accuracy")
predictions = [0, 1, 1, 0]
references = [0, 1, 0, 0]
result = accuracy.compute(predictions=predictions, references=references)
print(f"   Accuracy: {result}")

## 1.3 Huggingface Hub 탐색

Hub에서 모델을 탐색하고 정보를 가져오는 방법을 알아봅시다:

In [ ]:
from huggingface_hub import HfApi, list_models

# Hub API 초기화
api = HfApi()

# 인기 있는 한국어 모델 검색
print("🔍 한국어 BERT 모델 검색:\n")
korean_models = list(list_models(
    filter="bert",
    search="korean",
    limit=5,
    sort="downloads",
    direction=-1
))

for i, model in enumerate(korean_models, 1):
    print(f"{i}. {model.modelId}")
    print(f"   Downloads: {model.downloads:,}")
    print(f"   Likes: {model.likes}")
    print()

---
# **2. 기본 코드 구조 이해**

> **학습 목표**:
> - 모델 불러오기 방법 이해
> - 사전 학습 모델과 체크포인트 개념 이해

## 2.1 모델 불러오기의 기본 개념

### **체크포인트(Checkpoint)란?**

- 학습된 모델의 가중치와 설정이 저장된 상태
- Hub에 업로드된 모델 식별자 (예: `bert-base-uncased`)
- 로컬 경로에 저장된 모델 파일들

### **모델 구성 요소**

1. **Config**: 모델 아키텍처 정보 (`config.json`, `generation_config.json`)
2. **Weights**: 학습된 가중치 (`pytorch_model.bin`, `model-0000N-of-00002.safetensors`)
3. **Tokenizer**: 토크나이저 파일들 (`tokenizer_config.json`, `vocab.txt` 등)

In [ ]:
from transformers import (
    AutoModel, 
    AutoTokenizer, 
    AutoConfig,
    BertModel,
    BertTokenizer,
    BertConfig
)

print("🔄 모델 로딩 방법 비교\n")
print("="*60)

# 방법 1: Auto Classes (권장)
print("\n방법 1: Auto Classes 사용 (권장)")
model_name = "bert-base-uncased"
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

print(f"✅ Model type: {model.__class__.__name__}")
print(f"✅ Config type: {config.__class__.__name__}")
print(f"✅ Tokenizer type: {tokenizer.__class__.__name__}")

# 방법 2: 특정 모델 클래스 직접 사용
print("\n방법 2: 특정 모델 클래스 직접 사용")
bert_config = BertConfig.from_pretrained(model_name)
bert_tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

print(f"✅ Model type: {bert_model.__class__.__name__}")
print(f"✅ 동일한 모델: {type(model) == type(bert_model)}")

## 2.2 체크포인트 상세 정보 확인

In [ ]:
# 모델 설정 정보 확인
model_name = "bert-base-uncased"
config = AutoConfig.from_pretrained(model_name)

print("📋 BERT 모델 설정 (Config) 정보:\n")
print(f"아키텍처 타입: {config.model_type}")
print(f"히든 크기: {config.hidden_size}")
print(f"레이어 수: {config.num_hidden_layers}")
print(f"어텐션 헤드 수: {config.num_attention_heads}")
print(f"Vocabulary 크기: {config.vocab_size}")
print(f"최대 시퀀스 길이: {config.max_position_embeddings}")
print(f"Dropout 확률: {config.hidden_dropout_prob}")
print(f"\n전체 설정 키: {list(config.to_dict().keys())[:10]}...")

In [ ]:
# 모델 아키텍처 구조 확인
model = AutoModel.from_pretrained(model_name)

print("🏗️ 모델 아키텍처 구조:\n")
print(f"총 파라미터 수: {model.num_parameters():,}")
print(f"학습 가능한 파라미터: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# 모델의 주요 컴포넌트
print("\n주요 컴포넌트:")
for name, module in model.named_children():
    print(f"- {name}: {module.__class__.__name__}")
    if hasattr(module, 'num_parameters'):
        print(f"  파라미터 수: {module.num_parameters():,}")

# 모델의 전체 상세 구조
print("\n모델 구조:")
print(model)

## 2.3 모델 캐싱과 저장

In [ ]:
import os
from transformers import AutoModel, AutoTokenizer

# 캐시 디렉토리 확인
cache_dir = transformers.utils.hub.TRANSFORMERS_CACHE
print(f"📁 Transformers 캐시 디렉토리: {cache_dir}\n")

# 모델을 특정 디렉토리에 저장
save_directory = "./my_bert_model"
model_name = "bert-base-uncased"

# 모델과 토크나이저 로드
print("1. 모델 다운로드 및 로드...")
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 로컬에 저장
print(f"2. 모델을 '{save_directory}'에 저장...")
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

# 저장된 파일 확인
print(f"\n3. 저장된 파일들:")
if os.path.exists(save_directory):
    for file in os.listdir(save_directory):
        file_path = os.path.join(save_directory, file)
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        print(f"   - {file}: {size_mb:.2f} MB")

# 로컬에서 다시 로드
print(f"\n4. 로컬에서 모델 로드...")
local_model = AutoModel.from_pretrained(save_directory)
local_tokenizer = AutoTokenizer.from_pretrained(save_directory)
print("✅ 로컬 모델 로드 성공!")

In [ ]:
# 정리
import shutil
if os.path.exists(save_directory):
    shutil.rmtree(save_directory)
    print(f"\n🧹 임시 디렉토리 '{save_directory}' 삭제 완료")

---
# **3. Tokenizer 사용법**

> **학습 목표**:
> - 토크나이징 과정 및 주요 옵션 이해
> - 토크나이저와 모델의 연결 방식 이해

## 3.1 토크나이징 과정 이해

### **토크나이징의 핵심 단계**

1. **Text Normalization**: 텍스트 정규화 (소문자 변환 등)
2. **Pre-tokenization**: 공백 기준 분리
3. **Tokenization**: 서브워드 단위로 분할
4. **Post-processing**: 특수 토큰 추가

In [ ]:
from transformers import AutoTokenizer

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 예제 텍스트
text = "Tokenization is the first step in NLP!"

print("🔤 토크나이징 단계별 분석\n")
print("="*60)
print(f"원본 텍스트: '{text}'\n")

# Step 1: 토큰화 (특수 토큰 없이)
tokens = tokenizer.tokenize(text)
print("Step 1 - 토큰화:")
print(f"  토큰: {tokens}")
print(f"  토큰 수: {len(tokens)}\n")

# Step 2: 토큰을 ID로 변환
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Step 2 - 토큰 → ID 변환:")
for token, id in zip(tokens, token_ids):
    print(f"  '{token}' → {id}")

# Step 3: 인코딩 (특수 토큰 포함)
print("\nStep 3 - 전체 인코딩 (특수 토큰 포함):")
encoded = tokenizer.encode(text, add_special_tokens=True)
print(f"  인코딩된 ID: {encoded}")
print(f"  길이: {len(encoded)}")

# 특수 토큰 확인
print("\n특수 토큰:")
print(f"  [CLS] token ID: {tokenizer.cls_token_id}")
print(f"  [SEP] token ID: {tokenizer.sep_token_id}")

# 디코딩
decoded = tokenizer.decode(encoded)
print(f"\n디코딩 결과: '{decoded}'")

## 3.2 토크나이저 주요 옵션

In [ ]:
# 다양한 토크나이저 옵션 실습
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 테스트 문장들 (다양한 길이)
sentences = [
    "Short text.",
    "This is a medium length sentence for testing tokenizer.",
    "This is a very long sentence that will demonstrate how padding and truncation work in the tokenizer when we have different length inputs."
]

print("⚙️ 토크나이저 옵션 실습\n")

In [ ]:
# 옵션 1: padding
print("\n1. Padding 옵션:")
print("-"*40)

# padding=True (최대 길이에 맞춤)
encoded_padding = tokenizer(
    sentences,
    padding=True,
    return_tensors="pt"
)

for i, sent in enumerate(sentences):
    print(f"\n문장 {i+1} (길이: {len(sent.split())} 단어)")
    print(f"Input Tokens: {tokenizer.convert_ids_to_tokens(encoded_padding['input_ids'][i].tolist())}")
    print(f"Input IDs: {encoded_padding['input_ids'][i].tolist()}")
    print(f"Input IDs 길이: {len(encoded_padding['input_ids'][i])}")
    print(f"Attention Mask: {encoded_padding['attention_mask'][i].tolist()}")

In [ ]:
# 옵션 2: truncation
print("2. Truncation 옵션:")
print("-"*40)

max_length = 10
encoded_truncation = tokenizer(
    sentences[2],  # 긴 문장 사용
    truncation=True,
    max_length=max_length,
    return_tensors="pt"
)

print(f"원본 문장 토큰 수: {len(tokenizer.tokenize(sentences[2]))}")
print(f"Truncation 후 길이: {len(encoded_truncation['input_ids'][0])}")
print(f"Truncated IDs: {encoded_truncation['input_ids'][0].tolist()}")

In [ ]:
# 옵션 3: return_tensors
print("3. Return Tensors 옵션:")
print("-"*40)

# PyTorch tensors
encoded_pt = tokenizer(sentences[0], return_tensors="pt")
print(f"PyTorch tensor: {type(encoded_pt['input_ids'])}")
print(f"Shape: {encoded_pt['input_ids'].shape}")

# NumPy arrays
encoded_np = tokenizer(sentences[0], return_tensors="np")
print(f"NumPy array: {type(encoded_np['input_ids'])}")
print(f"Shape: {encoded_np['input_ids'].shape}")

# Python lists (default)
encoded_list = tokenizer(sentences[0])
print(f"Python list: {type(encoded_list['input_ids'])}")

## 3.3 토크나이저와 모델의 연결

In [ ]:
from transformers import AutoTokenizer, AutoModel

print("🔗 토크나이저와 모델 연결\n")
print("="*60)

In [ ]:
# 같은 체크포인트에서 토크나이저와 모델 로드
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 토크나이저와 모델의 vocabulary 크기 확인
print(f"토크나이저 vocabulary 크기: {tokenizer.vocab_size}")
print(f"모델 embedding 크기: {model.config.vocab_size}")
print(f"일치 여부: {tokenizer.vocab_size == model.config.vocab_size}\n")

In [ ]:
# 실제 사용 예제
text = ["Understanding the connection between tokenizer and model.", 
        "Understanding the connection between tokenizer and model. "*100]

# 토크나이징
inputs = tokenizer(
    text,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=64
)

print(inputs)


In [ ]:
print("토크나이저 출력:")
print(f"  - input_ids shape: {inputs['input_ids'].shape}")
print(f"  - attention_mask shape: {inputs['attention_mask'].shape}")
print(f"  - first sentence input_ids: {inputs['input_ids'][0]}")
print(f"  - first sentence attention_mask: {inputs['attention_mask'][0]}")
print(f"  - second sentence input_ids: {inputs['input_ids'][1]}")
print(f"  - second sentence original token length: {len(tokenizer.tokenize(text[1]))}")

In [ ]:
# 모델에 입력
with torch.no_grad():
    outputs = model(**inputs)

print(outputs.keys())

print("\n모델 출력:")
print(f"  - last_hidden_state shape: {outputs.last_hidden_state.shape}")
print(f"    (batch_size={outputs.last_hidden_state.shape[0]}, ")
print(f"     sequence_length={outputs.last_hidden_state.shape[1]}, ")
print(f"     hidden_size={outputs.last_hidden_state.shape[2]})")
print(f"  - pooler_output shape: {outputs.pooler_output.shape}")

In [ ]:
# 상황에 따라 다양한 출력을 획득할 수 있음
with torch.no_grad():
    outputs = model(**inputs, output_hidden_states=True, output_attentions=True)

print(outputs.keys())

print("\n모델 출력:")
print(f"  - last_hidden_state shape: {outputs.last_hidden_state.shape}")
print(f"    (batch_size={outputs.last_hidden_state.shape[0]}, ")
print(f"     sequence_length={outputs.last_hidden_state.shape[1]}, ")
print(f"     hidden_size={outputs.last_hidden_state.shape[2]})")
print(f"  - pooler_output shape: {outputs.pooler_output.shape}")
print(f"  - attentions shape: {outputs.attentions[0].shape}")
print(f"  - hidden_states shape: {outputs.hidden_states[0].shape}")


## 3.4 특수 토큰 커스터마이징

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 현재 특수 토큰들
print("기본 특수 토큰:")

for name, token in tokenizer.special_tokens_map.items():
    if token:
        token_id = tokenizer.convert_tokens_to_ids(token)
        print(f"  {name:4} token: '{token}' (ID: {token_id})")

In [ ]:
# 문장 쌍 인코딩 (BERT의 경우)
print("\n문장 쌍 인코딩 예제:")
sentence1 = "This is the first sentence."
sentence2 = "This is the second sentence."

# 두 문장을 함께 인코딩
encoded = tokenizer(
    sentence1,
    sentence2,
    return_tensors="pt"
)

# token_type_ids 확인 (문장 구분)
print(f"\nToken IDs: {encoded['input_ids'][0].tolist()}")
print(f"Token Type IDs: {encoded['token_type_ids'][0].tolist()}")
print(f"  → 0: 첫 번째 문장, 1: 두 번째 문장")

In [ ]:
# 디코딩해서 확인
decoded = tokenizer.decode(encoded['input_ids'][0])
print(f"\n디코딩 결과: '{decoded}'")

---
# **4. 대표적인 모델 클래스 소개**

> **학습 목표**:
> - SequenceClassification, CausalLM 등 태스크별 클래스 구조 이해
> - forward 메서드의 주요 파라미터와 반환값 분석

## 4.1 태스크별 모델 클래스 개요

### **주요 모델 클래스들**

Huggingface는 각 태스크에 특화된 모델 클래스를 제공합니다:

1. **AutoModel**: Base model (hidden states만 반환)
2. **AutoModelForSequenceClassification**: 텍스트 분류
3. **AutoModelForTokenClassification**: 토큰 분류 (NER, POS)
4. **AutoModelForQuestionAnswering**: 질의응답
5. **AutoModelForCausalLM**: 텍스트 생성 (GPT류)
6. **AutoModelForMaskedLM**: Masked Language Modeling (BERT류)
7. **AutoModelForSeq2SeqLM**: Sequence-to-Sequence (T5, BART)

In [ ]:
print("📚 태스크별 모델 클래스 비교\n")
print("="*70)

from transformers import (
    # AutoModel,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    AutoModelForQuestionAnswering,
    AutoModelForCausalLM,
    AutoModelForMaskedLM
)

In [ ]:
# 각 태스크에 적합한 모델 예시
model_examples = [
    ("Sequence Classification", "distilbert-base-uncased-finetuned-sst-2-english", AutoModelForSequenceClassification),
    ("Token Classification", "dbmdz/bert-large-cased-finetuned-conll03-english", AutoModelForTokenClassification),
    ("Question Answering", "distilbert-base-cased-distilled-squad", AutoModelForQuestionAnswering),
    ("Causal LM", "gpt2", AutoModelForCausalLM),
    ("Masked LM", "bert-base-uncased", AutoModelForMaskedLM)
]

for task, model_name, model_class in model_examples:
    print(f"\n{'='*50}")
    print(f"태스크: {task}")
    print(f"모델: {model_name}")
    print(f"클래스: {model_class.__name__}")
    
    # 모델 로드 (실제로는 로드하지 않고 config만 확인)
    config = AutoConfig.from_pretrained(model_name)
    
    print(f"아키텍처: {config.architectures[0] if hasattr(config, 'architectures') else 'N/A'}")
    
    # 태스크별 특수 속성
    if hasattr(config, 'num_labels'):
        print(f"Number of labels: {config.num_labels}")
    if hasattr(config, 'id2label'):
        print(f"Labels: {list(config.id2label.values())[:5]}...")

## 4.2 SequenceClassification 모델 상세

In [ ]:
# 감성 분석 모델 로드
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# 모델 개요
print("모델 개요:")
print(f"- Base model: {model.base_model_prefix}")
print(f"- Number of labels: {model.config.num_labels}")
print(f"- Labels: {model.config.id2label}\n")

In [ ]:
# 모델 상세 구조
print("모델 상세 구조:")
print(model)

In [ ]:
# Forward 메서드 파라미터 테스트 (Positive 예시)
text = "This movie is absolutely fantastic!"
inputs = tokenizer(text, return_tensors="pt")

print(f"입력 텍스트: '{text}'\n")
print("Forward 메서드 입력 파라미터:")
for key, value in inputs.items():
    print(f"  - {key}: shape {value.shape}")

# Forward pass
with torch.no_grad():
    outputs = model(**inputs)

In [ ]:
# 출력 파라미터
print(outputs.keys())

In [ ]:
# 출력 분석
print("\nForward 메서드 출력:")
print(f"  - logits: shape {outputs.logits.shape}")
print(f"    값: {outputs.logits[0].tolist()}")

In [ ]:
# Forward 메서드 파라미터 테스트 (Negative 예시)
text = "This movie is totally awful!"
inputs = tokenizer(text, return_tensors="pt")

print(f"입력 텍스트: '{text}'\n")
print("Forward 메서드 입력 파라미터:")
for key, value in inputs.items():
    print(f"  - {key}: shape {value.shape}")

# Forward pass
with torch.no_grad():
    outputs = model(**inputs)

In [ ]:
# 출력 분석
print("\nForward 메서드 출력:")
print(f"  - logits: shape {outputs.logits.shape}")
print(f"    값: {outputs.logits[0].tolist()}")

In [ ]:
# Hidden states 접근 (output_hidden_states=True)
with torch.no_grad():
    outputs_with_hidden = model(**inputs, output_hidden_states=True)

if outputs_with_hidden.hidden_states:
    print(f"\n  - hidden_states: {len(outputs_with_hidden.hidden_states)} layers")
    print(f"    각 레이어 shape: {outputs_with_hidden.hidden_states[0].shape}")

# Attention weights 접근 (output_attentions=True)
with torch.no_grad():
    outputs_with_attention = model(**inputs, output_attentions=True)

if outputs_with_attention.attentions:
    print(f"\n  - attentions: {len(outputs_with_attention.attentions)} layers")
    print(f"    각 attention shape: {outputs_with_attention.attentions[0].shape}")
    print(f"    (batch_size, num_heads, seq_length, seq_length)")

In [ ]:
# 만약, SequenceClassification 모델이 아닌 경우?
model_ = AutoModelForSequenceClassification.from_pretrained("klue/roberta-large")

In [ ]:
print(model_)

In [ ]:
# Forward가 될까...?
text = "This movie is totally awful!"
inputs = tokenizer(text, return_tensors="pt")

print(f"입력 텍스트: '{text}'\n")
print("Forward 메서드 입력 파라미터:")
for key, value in inputs.items():
    print(f"  - {key}: shape {value.shape}")

# Forward pass
with torch.no_grad():
    outputs = model_(**inputs)

In [ ]:
# 출력을 확인해보자
print("\nForward 메서드 출력:")
print(f"  - logits: shape {outputs.logits.shape}")
print(f"    값: {outputs.logits[0].tolist()}")

## 4.3 CausalLM 모델 상세

In [ ]:
# GPT-2 모델 로드
model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# 모델 개요
print("모델 개요:")
print(f"- Model type: {model.config.model_type}")
print(f"- Vocab size: {model.config.vocab_size}")
print(f"- Max position embeddings: {model.config.n_positions}")
print(f"- Number of layers: {model.config.n_layer}")
print(f"- Hidden size: {model.config.hidden_size}") 

In [ ]:
# 모델 상세 구조
print("모델 상세 구조:")
print(model)

In [ ]:
# 텍스트 생성을 위한 forward pass
prompt = "The future of AI is going to be very"
inputs = tokenizer(prompt, return_tensors="pt")

print(f"프롬프트: '{prompt}'\n")
print(f"프롬프트 토큰 길이: {len(tokenizer.tokenize(prompt))}")

# Forward pass
with torch.no_grad():
    outputs = model(**inputs)

In [ ]:
# 출력 파라미터
print(outputs.keys())

In [ ]:
print("Forward 메서드 출력:")
print(f"  - logits shape: {outputs.logits.shape}")
print(f"    (batch_size, sequence_length, vocab_size)")

In [ ]:
# 다음 토큰 예측
next_token_logits = outputs.logits[0, -1, :]  # 마지막 토큰의 logits
next_token_probs = torch.softmax(next_token_logits, dim=-1)

In [ ]:
# 상위 5개 가능한 다음 토큰
top_k = 5
top_probs, top_indices = torch.topk(next_token_probs, top_k)

print(f"\n다음 토큰 예측 (Top {top_k}):")
for i in range(top_k):
    token = tokenizer.decode(top_indices[i])
    prob = top_probs[i].item()
    print(f"  {i+1}. '{token}' (확률: {prob:.3f})")

In [ ]:
# Labels를 사용한 Loss 계산

# 입력과 동일한 labels 생성 (teacher forcing)
prompt = "The future of AI is going to be very interesting. With transformers, it will be even more interesting."
inputs = tokenizer(prompt, return_tensors="pt")

labels = inputs['input_ids'].clone()

with torch.no_grad():
    outputs_with_loss = model(**inputs, labels=labels, outputs_with_loss=True)

In [ ]:
# 각 위치에서의 Top-k 토큰 예측

next_tokens_logits = outputs_with_loss.logits[0, :, :]  # 마지막 토큰의 logits
next_tokens_probs = torch.softmax(next_tokens_logits, dim=-1)

top_k = 5
top_probs, top_indices = torch.topk(next_tokens_probs, top_k)

for i in range(len(next_tokens_logits)-1):
    print(f"현재 Input Text: {tokenizer.decode(inputs['input_ids'][0][:i+1])}")
    print(f"실제 정답: {tokenizer.decode(labels[0][i+1])}")
    for j in range(top_k):
        print(f"  {i+1}. '{tokenizer.decode(top_indices[i][j])}' (확률: {top_probs[i][j]:.3f})")
    print("-"*40)


In [ ]:
print(f"  - Loss: {outputs_with_loss.loss.item():.4f}")
print(f"  - Perplexity: {torch.exp(outputs_with_loss.loss).item():.2f}")

## 4.4 모델 클래스 비교 및 선택 가이드

In [ ]:
print("📊 모델 클래스 선택 가이드\n")
print("="*70)

# 태스크별 모델 선택 가이드
task_guide = {
    "감성 분석 / 텍스트 분류": {
        "class": "AutoModelForSequenceClassification",
        "추천 모델": ["bert-base-uncased", "roberta-base", "distilbert-base-uncased"],
        "출력": "logits (num_labels 차원)",
        "용도": "스팸 필터링, 감성 분석, 주제 분류"
    },
    "개체명 인식 / POS 태깅": {
        "class": "AutoModelForTokenClassification",
        "추천 모델": ["bert-base-cased", "roberta-base"],
        "출력": "각 토큰별 logits",
        "용도": "NER, POS tagging, 구문 분석"
    },
    "텍스트 생성": {
        "class": "AutoModelForCausalLM",
        "추천 모델": ["gpt2", "gpt2-medium", "gpt2-large"],
        "출력": "다음 토큰 예측 logits",
        "용도": "챗봇, 스토리 생성, 코드 생성"
    },
    "질의응답": {
        "class": "AutoModelForQuestionAnswering",
        "추천 모델": ["bert-base-uncased", "roberta-base"],
        "출력": "start/end position logits",
        "용도": "문서 내 답변 추출, FAQ 시스템"
    },
    "빈칸 채우기": {
        "class": "AutoModelForMaskedLM",
        "추천 모델": ["bert-base-uncased", "roberta-base"],
        "출력": "마스크 토큰 위치의 vocab logits",
        "용도": "문장 완성, 단어 예측"
    },
}

for task, info in task_guide.items():
    print(f"\n📌 {task}")
    print(f"   클래스: {info['class']}")
    print(f"   추천 모델: {', '.join(info['추천 모델'][:2])}")
    print(f"   출력 형태: {info['출력']}")
    print(f"   사용 예: {info['용도']}")

---
# **5. 추론 및 결과 확인**

> **학습 목표**:
> - 로짓(logits) 해석 및 후처리(softmax, argmax 등) 이해
> - 실제 추론 파이프라인 구현

## 5.1 Logits 이해와 변환

### **Logits란?**
- 모델의 최종 레이어 출력 (활성화 함수 적용 전)
- 확률이 아닌 원시 점수(raw scores)
- Softmax를 통해 확률로 변환 가능

In [ ]:
# 모델과 토크나이저 로드
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)
tokenizer = AutoTokenizer.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

# 테스트 문장들
test_sentences = [
    "This product is amazing!",
    "Terrible experience.",
    "I think",
]

# 토크나이징
inputs = tokenizer(test_sentences, return_tensors="pt", padding=True)

# 추론
with torch.no_grad():
    outputs = model(**inputs)

In [ ]:
print(f"출력 종류: {outputs.keys()}")
print(f"출력 크기: {outputs.logits.shape}")

In [ ]:
for sentence, logits in zip(test_sentences, outputs.logits):
    print(f"\n입력: '{sentence}'")
    print("\n")
    
    # 1. Raw logits
    print(f"1. Raw Logits: {logits.tolist()}")
    print(f"   → NEGATIVE={logits[0]:.3f}, POSITIVE={logits[1]:.3f}")
    
    # 2. Softmax 확률
    probs = F.softmax(logits, dim=-1)
    print(f"\n2. Softmax 확률: {probs.tolist()}")
    print(f"   → NEGATIVE={probs[0]:.3%}, POSITIVE={probs[1]:.3%}")
    
    # 3. Argmax 예측
    prediction = torch.argmax(logits)
    labels = ["NEGATIVE", "POSITIVE"]
    print(f"\n3. 최종 예측: {labels[prediction]} (인덱스: {prediction})")
    
    # 4. Confidence score
    confidence = probs.max().item()
    print(f"\n4. Confidence: {confidence:.3%}")
    print("\n")
    print("="*40)

## 5.2 배치 추론과 최적화

In [ ]:
# 모델과 토크나이저 준비
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)
tokenizer = AutoTokenizer.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

# 테스트 데이터 생성
test_texts = [
    "Great product, highly recommend! Great product, highly recommend!Great product, highly recommend!Great product, highly recommend!Great product, highly recommend!",
    "Not worth the price.",
    "Average quality, nothing special.",
    "Exceeded my expectations!",
    "Disappointed with the purchase.",
    "Good value for money.",
    "Would buy again.",
    "Complete waste of money."
] * 5  # 40개 문장

print(f"테스트 데이터: {len(test_texts)}개 문장\n")

In [ ]:
import time

print("⚡ 배치 추론 성능 비교\n")
print("="*60)

# 1. 개별 추론
print("1. 개별 추론 (one-by-one):")
start_time = time.time()

individual_results = []
for text in test_texts:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = F.softmax(outputs.logits, dim=-1)
    individual_results.append(probs)

individual_time = time.time() - start_time
print(f"   소요 시간: {individual_time:.3f}초")
print(f"   평균 처리 시간: {individual_time/len(test_texts)*1000:.2f}ms/문장")

# 2. 배치 추론
print("\n2. 배치 추론 (batch processing):")
start_time = time.time()

# 배치로 토크나이징
inputs = tokenizer(
    test_texts,
    return_tensors="pt",
    padding=True,
    truncation=True
)

# 배치 추론
with torch.no_grad():
    outputs = model(**inputs)
batch_probs = F.softmax(outputs.logits, dim=-1)

batch_time = time.time() - start_time
print(f"   소요 시간: {batch_time:.3f}초")
print(f"   평균 처리 시간: {batch_time/len(test_texts)*1000:.2f}ms/문장")

# 성능 비교
print(f"\n📊 성능 향상: {individual_time/batch_time:.2f}배 빠름")

# **수고하셨습니다! 🎉**